In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio.v2 as imageio
import io

fulldatasetpath = '/Volumes/Data_Drive/datasets/VIDIMU'
file_path = os.path.join(fulldatasetpath,'dataset','videoandimusyncrop/S40/S40_A01_T01.csv')
acts = ['A01', 'A02', 'A03', 'A04', 'A05', 'A07', 'A09', 'A10', 'A11', 'A13']
trs = ['T01', 'T01', 'T02', 'T02', 'T01', 'T01', 'T02', 'T01', 'T01', 'T01']

paths = [os.path.join(fulldatasetpath,'dataset',f'videoandimusyncrop/S40/S40_{acts[i]}_{trs[i]}.csv') for i in range(len(acts))]
print(paths)

['/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A01_T01.csv', '/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A02_T01.csv', '/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A03_T02.csv', '/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A04_T02.csv', '/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A05_T01.csv', '/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A07_T01.csv', '/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A09_T02.csv', '/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A10_T01.csv', '/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A11_T01.csv', '/Volumes/Data_Drive/datasets/VIDIMU/dataset/videoandimusyncrop/S40/S40_A13_T01.csv']


In [2]:
df = pd.read_csv(file_path)
df = df.iloc[:,1:]
print(df.columns[:3])
joint_names_updated = [df.columns[j][:-2] for j in range(1, 103, 3)]

Index(['pelvis_x', 'pelvis_y', 'pelvis_z'], dtype='object')


In [3]:
print(joint_names_updated)

['pelvis', 'left_hip', 'right_hip', 'torso', 'left_knee', 'right_knee', 'neck', 'left_ankle', 'right_ankle', 'left_big_toe', 'right_big_toe', 'left_small_toe', 'right_small_toe', 'left_heel', 'right_heel', 'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky_knuckle', 'right_pinky_knuckle', 'left_middle_tip', 'right_middle_tip', 'left_index_knuckle', 'right_index_knuckle', 'left_thumb_tip', ' right_thumb_tip']


## Normalize

In [4]:
import numpy as np
import pandas as pd

def normalize_joint_positions(joint_data, root_joint='pelvis', epsilon=1e-6):
    """
    Normalize joint positions using a root alignment and local coordinate transformation.

    Parameters:
    - joint_data: DataFrame containing the joint positions (columns: 'joint_x', 'joint_y', 'joint_z')
    - root_joint: The joint to be used as the reference root
    - epsilon: Small value to avoid division by zero

    Returns:
    - normalized_data: DataFrame with normalized joint positions
    """

    root_positions = joint_data[[f'{root_joint}_x', f'{root_joint}_y', f'{root_joint}_z']].values
    normalized_data = joint_data.copy()

    for joint in joint_names_updated:
        normalized_data[[f'{joint}_x']] -= root_positions[:, 0][:, None]
        normalized_data[[f'{joint}_y']] -= root_positions[:, 1][:, None]
        normalized_data[[f'{joint}_z']] -= root_positions[:, 2][:, None]

    pelvis_abs_positions = pd.DataFrame({
        'pelvis_abs_x': root_positions[:, 0],
        'pelvis_abs_y': root_positions[:, 1],
        'pelvis_abs_z': root_positions[:, 2]
    })

    normalized_data = pd.concat([normalized_data, pelvis_abs_positions], axis=1)

    return normalized_data


## Plot

In [ ]:
def plot_skeleton_frame_with_axes(ax, frame_idx, normalized_df, joint_coords_updated):
    joints = {joint: joint_coords_updated[joint][frame_idx] for joint in joint_names_updated}

    x_min, x_max = normalized_df[[f'{joint}_x' for joint in joint_names_updated]].min().min(), normalized_df[[f'{joint}_x' for joint in joint_names_updated]].max().max()
    y_min, y_max = normalized_df[[f'{joint}_y' for joint in joint_names_updated]].min().min(), normalized_df[[f'{joint}_y' for joint in joint_names_updated]].max().max()
    z_min, z_max = normalized_df[[f'{joint}_z' for joint in joint_names_updated]].min().min(), normalized_df[[f'{joint}_z' for joint in joint_names_updated]].max().max()

    connections_updated = [
        ('pelvis', 'left_hip'), ('pelvis', 'right_hip'),
        ('left_hip', 'left_knee'), ('right_hip', 'right_knee'),
        ('left_knee', 'left_ankle'), ('right_knee', 'right_ankle'),
        ('left_ankle', 'left_heel'), ('right_ankle', 'right_heel'),
        ('left_heel', 'left_big_toe'), ('right_heel', 'right_big_toe'),
        ('left_heel', 'left_small_toe'), ('right_heel', 'right_small_toe'),
        ('pelvis', 'torso'), ('torso', 'neck'), ('neck', 'nose'),
        ('neck', 'left_shoulder'), ('neck', 'right_shoulder'),
        ('left_shoulder', 'left_elbow'), ('right_shoulder', 'right_elbow'),
        ('left_elbow', 'left_wrist'), ('right_elbow', 'right_wrist'),
        ('left_wrist', 'left_pinky_knuckle'), ('right_wrist', 'right_pinky_knuckle'),
        ('left_wrist', 'left_index_knuckle'), ('right_wrist', 'right_index_knuckle'),
        ('left_wrist', 'left_thumb_tip'), ('right_wrist', 'right_thumb_tip')
    ]

    xs, ys, zs = [], [], []
    for joint in joint_names_updated:
        if joint in joints: 
            x, y, z = joints[joint]
            xs.append(x)
            ys.append(y)
            zs.append(z)

    ax.scatter(xs, ys, zs, color='blue')
    for conn in connections_updated:
        if conn[0] in joints and conn[1] in joints: 
            x_vals = [joints[conn[0]][0], joints[conn[1]][0]]
            y_vals = [joints[conn[0]][1], joints[conn[1]][1]]
            z_vals = [joints[conn[0]][2], joints[conn[1]][2]]
            ax.plot(x_vals, y_vals, z_vals, color='red')

    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    ax.set_zlim([z_min, z_max])

    ax.grid(True)
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')

def output_gif(normalized_df, pth):
    joint_coords_updated = {}
    for joint in joint_names_updated:
        joint_coords_updated[joint] = normalized_df[[f'{joint}_x', f'{joint}_y', f'{joint}_z']].values

    gif_images_dynamic = []
    n_frames = 5 # 50/n_frames must be integer [1, 2, 5, 10]
    for frame in range(0, normalized_df.shape[0], n_frames): 
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection='3d')

        ax.view_init(elev=-45, azim=-90)  
        plot_skeleton_frame_with_axes(ax, frame, normalized_df, joint_coords_updated)

        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        gif_images_dynamic.append(imageio.imread(buf))
        buf.close()
        plt.close(fig)  

    imageio.mimsave(pth, gif_images_dynamic, fps=50/n_frames)

def output_gif(normalized_df, pth):
    joint_coords_updated = {}
    for joint in joint_names_updated:
        joint_coords_updated[joint] = normalized_df[[f'{joint}_x', f'{joint}_y', f'{joint}_z']].values

    gif_images_dynamic = []
    n_frames = 10  # Skip every 10th frame to reduce the GIF size
    for frame in range(0, normalized_df.shape[0], n_frames):
        # Use a smaller figure size to reduce resolution
        fig = plt.figure(figsize=(4, 3))  # Adjust dimensions to lower resolution
        ax = fig.add_subplot(111, projection='3d')

        # Set the view for consistency
        ax.view_init(elev=-45, azim=-90)
        plot_skeleton_frame_with_axes(ax, frame, normalized_df, joint_coords_updated)

        # Save the frame to a buffer
        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=80)  # Lower dpi for reduced resolution
        buf.seek(0)
        gif_images_dynamic.append(imageio.imread(buf))  # Append the frame to the GIF
        buf.close()
        plt.close(fig)

    # Save the GIF with reduced frame rate
    imageio.mimsave(pth, gif_images_dynamic, fps=50/n_frames)  # Lower FPS to reduce file size

In [6]:
normalized_df = normalize_joint_positions(df, root_joint='pelvis')

print(normalized_df[['pelvis_abs_x', 'pelvis_abs_y', 'pelvis_abs_z']].head())
print(f"Number of columns: {len(normalized_df.columns)}")

for i, path in enumerate(paths):
    df = pd.read_csv(path)
    df = df.iloc[:,1:]
    normalized_df = normalize_joint_positions(df, root_joint='pelvis')
    filename = f'skeleton_3d_normalized_{acts[i]}_{trs[i]}.gif'
    outpth = os.path.join('outs', filename)
    output_gif(normalized_df, outpth)
    print(f'Processed {acts[i]}_{trs[i]}')

   pelvis_abs_x  pelvis_abs_y  pelvis_abs_z
0        1466.9         229.4        6657.7
1        1472.5         230.2        6683.8
2        1475.9         230.6        6700.3
3        1478.1         230.9        6711.0
4        1481.5         231.4        6726.2
Number of columns: 105
Processed A01_T01
Processed A02_T01
Processed A03_T02
Processed A04_T02
Processed A05_T01
Processed A07_T01
Processed A09_T02
Processed A10_T01
Processed A11_T01
Processed A13_T01
